### Predicting ATM Fraud

In this notebook you will learn how to build a classification model with Spark machine learning API (SparkML).

The notebook implements the following steps:
- Perform data understanding and preparation tasks
- Build a model with SparkML API
- Evaluate and test the model

### Use Case

The analytics use case implemented in this notebook is detecting ATM Fraud. While it's a simple use case, it implements all steps from the CRISP-DM methodolody, which is the recommended best practice in data science.
![CRISP-DM](https://raw.githubusercontent.com/rosswlewis/ATM_Fraud/master/assets/crisp_dm.png)

Use case implementation starts with defining the business problem and identifying the data that can be used to solve the problem. For ATM fraud, we use historical transaction data. We also know which transactions are fraud, which is the critical information for building predictive models. In the next step, we use visual APIs for data understanding and complete some data preparation tasks. In a typical analytics project data preparation will include more steps (for example, formatting data or deriving new variables). 

Once the data is ready, we can build a predictive model. In our example we show implementaiton of SparkML *Logistic Regression* and *Random Forrest* classification models. Classification is a statistical technique which assigns a "class" to each customer record (for our use case "fraud" or "not fraud"). Classification models use historical data to come up with the logic to predict "class", this process is called model training. After the model is created, it's usually evaluated using another data set. 

### Step 1: Load the Data

In [ ]:
# This code is generated by Watson Studio "Insert Spark Dataframe" function

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-210be2d3-af5e-43b6-9d74-0427ae8ae81a',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': '9FthwkkdExB6wrCH1HafvISYEZEeKAKEKpWfPg9TV2hM'
}

configuration_name = 'os_b6ce1d66a23747c685affa13595b2acb_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# We renamed the dataframe in the generated code to "historicalATMFraud"
historicalATMFraud = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(cos.url('ATM_Data.csv', 'pyrlabtest-donotdelete-pr-ecn1gj2kpu5dli'))
historicalATMFraud.take(5)


If the first step ran successfully (you saw the output), then continue reviewing the notebook and running each code cell step by step. Note that not every cell has a visual output. The cell is still running if you see a * in the brackets next to the cell. 

If the first step didn't finish successfully, check with the instructor. 

### Step 2: Data Understanding

In [ ]:
# The show funtion provides a better view of the data than the take() method
historicalATMFraud.show()

In [ ]:
# Let's rename some columns - they will display better
historicalATMFraud = historicalATMFraud.withColumnRenamed('POST_COD_Town','CITY')
historicalATMFraud = historicalATMFraud.withColumnRenamed('POST_COD_Region','REGION')
historicalATMFraud = historicalATMFraud.withColumnRenamed('Day of Week','DAY')
historicalATMFraud = historicalATMFraud.withColumnRenamed('Time of Day','TIME')
historicalATMFraud = historicalATMFraud.withColumnRenamed('Time of Day Band','TIME_BAND')

In [ ]:
# Next, we review the data types
historicalATMFraud.dtypes

If you used the generated code to read in data, then the data type for all variables is *string*. Some variables, for example, the FRAUD flag, should be an interger. We can convert data types, but if you have many columns, it may be easier to add the *inferSchema* parameter on data load.

The modified code to read in data will look like this: 

**historicalATMFraud = spark.read\ <BR>
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\ <BR>
  .option('header', 'true')\ <BR>
  .option('inferSchema', 'true')\ <BR>
  .load(cos.url('ATM_Data.csv', 'pyrlabtest-donotdelete-pr-ecn1gj2kpu5dli'))**
  
If you rerun the code to read in data, now **MODEL** and **Time of Day** columns have data type of *int*

In [ ]:
# Display summary statistics
historicalATMFraud.describe().show()

In [ ]:
# Summary statistics may be difficult to read in the row format, so let's display it in the column format
historicalATMFraud.describe().show(vertical=True)

Since most variables are non-numeric, we will do additional data understanding later. But the first descriptive statistics, **count**, shows us that we may have some invalid values in several columns. We can confirm the row count by using the count() funtion.  

In [ ]:
historicalATMFraud.count()

The output of the describe() method showed us that the following columns have less than 21750 records: **FRAUD, ISSUER_I, MAKE, MODEL, FACILITI, ATM_POSI, INSTITUT, CITY**. There are several ways to deal with missing values. Factors such as number of missing values and the importance of the field for modeling should be taken into consideration. 

Since we are focusing on building a model, and not on feature selection/engineering, we will use a simple approach:
- Drop the MODEL and ISSUER_I field (many missing values and not likely useful for prediciton of fraud)
- Drop rows with missing values for the rest of the fields

In [ ]:
# Drop columns which we will not use for data understanding and modeling
historicalATMFraud = historicalATMFraud.drop("ATM_ID","MAKE","MODEL","ISSUER_I")
# Display the count of the data frame if we drop the records
historicalATMFraud.na.drop().count()

In [ ]:
# Since data frames are not mutable, we have to create a new object with null values removed
historicalATMFraud.count()

In [ ]:
historicalATMFraud=historicalATMFraud.na.drop()
# Check record count of the new data frame
historicalATMFraud.count()

### Step 3: Understand the Data with Visualizations

Visualization is one of the most effective ways to understand data. There are several visualization APIs that can be used. In this sample notebook we will use *PixieDust*, which provides an interactive interface for visualizations. <BR>

PixieDust supports Spark data frames, while many other visualization APIs work only with Pandas data frames. <BR>

More information about PixieDust: https://github.com/ibm-cds-labs/pixiedust?cm_mc_uid=78151411419314871783930&cm_mc_sid_50200000=1487962969  <BR>


Using the interactive interface, check the number of fradulent and non-fradulent transactions. It's important that the dataset used for modeling is balanced. 


In [ ]:
# Run this line once in a notebook
#!pip install --upgrade pixiedust


from pixiedust.display import *
display(historicalATMFraud)

### Step 4: Build the Spark pipeline and the Random Forest models
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

We are not using all columns for modeling because we can tell that some of them are correlated just by description - *Region* and *City* (we kept *City*), and 
*Time* and *Time Band* (we kept *Time*)<BR>
As mentioned earlier, in a production implementation a data scientist will use statistical and other methods for feature selection. 

In [30]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler


# This code desribes the steps that should be applied to categorical data for modeling. First, we convert String values to an index, and then we encode them.
# The steps are recorded in teh "stages" object. The indexing/encoding is not applied until we run the Pipeline.fit() method.  
categoricalColumns = ["ATM_POSI", "DAY", "CITY", "SHORTNAM", "CARDHOLD", "FACILITI"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="FRAUD", outputCol="label")
stages += [label_stringIdx]


In [ ]:
# We also have one numeric column
numericCols = ["TIME"]

# The ML API requires that all features are passed in as a single Vector, which we are creating here
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
partialPipeline = Pipeline().setStages(stages)
# Running this code will apply the string indexing and encoding operations
pipelineModel = partialPipeline.fit(historicalATMFraud)
preppedDataDF = pipelineModel.transform(historicalATMFraud)

In [ ]:
# You can review the data that will be used for modeling
display(preppedDataDF)

In [ ]:
### Split data into training and test sets. Set seed for reproducibility
(trainingData, testData) = preppedDataDF.randomSplit([0.8, 0.2], seed=100)
trainingData.cache()
testData.cache()

In [ ]:
# First, we'll build a logistic regression model

from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [ ]:
# Make predictions on test data using the transform() method.
predictions = lrModel.transform(testData)
display(predictions)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model - the displayed evaluation is AUC ROC
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [ ]:
# Next, we will build a Random Forest model
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [ ]:
# Make predictions on test data using the Transformer.transform() method.
rf_predictions = rfModel.transform(testData)

# Evaluate model
rf_evaluator = BinaryClassificationEvaluator()
rf_evaluator.evaluate(rf_predictions)

**We have finished building and testing a predictive model.** <BR>
As shown in the diagram of the CRISP-DM process, a data scientist will usually spend more time on refining the model (testing with different features and algorithms). After the desired accuracy is achieved, a model is deployed in production. We will cover deployment in another lab. 